In [ ]:
"""
Greta Tengattini, 
Love,Indus

Introduction:
This script is designed to interface with the Facebook API by parsing HAR (HTTP Archive) files.
It extracts ad-related data from Facebook Meta's ad library, processes the relevant details, and exports the structured data into both CSV and Excel formats. 
By automating this process, we can save hours of manual work, ensuring efficiency and accuracy in ad data collection.
"""

# Import necessary libraries
from haralyzer import HarParser, HarPage  # HAR file parsing (not used directly but can be useful)
import json  # Handle JSON data
import os  # File path operations
import pandas as pd  # Data manipulation and storage
import datetime as dt  # Handling date and time
from bs4 import BeautifulSoup  # Parsing HTML content
import xlsxwriter  # Excel file handling
from datetime import datetime, timedelta  # Additional date utilities

# Define input and output file paths
name_string = 'Fansidea_com__20240325'  # Base name for the files
filename = os.path.join(r'C:\Users\inste\Downloads', name_string + '.har')  # HAR file path
output_filename = os.path.join(r'C:\Users\inste\Downloads', name_string + '.csv')  # CSV output path
xlsx_filename = os.path.join(r'C:\Users\inste\Downloads', name_string + '.xlsx')  # Excel output path

# Load HAR file and parse JSON
with open(filename, 'r', errors='replace') as f:
    har = json.loads(f.read())

# Initialize lists to store extracted ad data
entry_list = []  # List of processed entries
entry_counter = 0  # Track number of processed entries
ads = []  # Store extracted ad details

# Loop through each network request in the HAR file
for entry in har['log']['entries']:
    try:
        url = entry['request']['url']  # Extract request URL
        
        # Identify Facebook ad library requests
        if 'https://www.facebook.com/ads/library/async/search_ads/?' in url:
            entry_list.append(entry_counter)  # Track the entry index
            
            # Extract response data and clean up Facebook-specific prefixes
            items = json.loads(entry['response']['content']['text'].replace('for (;;);', ''))
            elements = items['payload']['results']  # Extract relevant ad data
            
            # Loop through extracted ads
            for element in elements:
                results = {
                    'adArchiveID': element[0]['adArchiveID'],  # Ad archive ID
                    'ad_link': f'https://www.facebook.com/ads/library/?id={element[0]["adArchiveID"]}',  # Direct link to ad
                    'startDate': dt.datetime.fromtimestamp(element[0]['startDate']).strftime('%Y-%m-%d'),  # Ad start date
                    'endDate': dt.datetime.fromtimestamp(element[0]['endDate']).strftime('%Y-%m-%d'),  # Ad end date
                    'pageID': element[0]['pageID'],  # Page ID running the ad
                    'pageName': element[0]['pageName'],  # Page name
                    'link_url': element[0]['snapshot']['link_url'],  # Ad link URL
                    'cta_text': element[0]['snapshot']['cta_text'],  # Call to action text
                }
                
                # Extract body text
                try:
                    results['body_text'] = element[0]['snapshot']['cards'][0]['body']
                except (KeyError, ValueError, IndexError):
                    try:
                        results['body_text'] = BeautifulSoup(element[0]['snapshot']['body']['markup']['__html'], features='html.parser').get_text('\n')
                    except (KeyError, ValueError, IndexError):
                        results['body_text'] = ''
                
                # Extract ad title
                try:
                    results['title'] = element[0]['snapshot']['cards'][0]['title']
                except (KeyError, ValueError, IndexError):
                    results['title'] = element[0]['snapshot'].get('title', '')
                
                # Extract ad caption
                try:
                    results['caption'] = element[0]['snapshot']['cards'][0]['caption']
                except (KeyError, ValueError, IndexError):
                    results['caption'] = element[0]['snapshot'].get('caption', '')
                
                # Extract ad link description
                try:
                    results['link_description'] = element[0]['snapshot']['cards'][0]['link_description']
                except (KeyError, ValueError, IndexError):
                    results['link_description'] = element[0]['snapshot'].get('link_description', '')
                
                # Extract image URL
                try:
                    results['original_image_url'] = element[0]['snapshot']['cards'][0]['original_image_url']
                except (KeyError, ValueError, IndexError):
                    results['original_image_url'] = element[0]['snapshot'].get('images', [{}])[0].get('original_image_url', '')
                
                # Extract video preview image URL
                try:
                    results['video_preview_image_url'] = element[0]['snapshot']['cards'][0]['video_preview_image_url']
                except (KeyError, ValueError, IndexError):
                    results['video_preview_image_url'] = element[0]['snapshot'].get('videos', [{}])[0].get('video_preview_image_url', '')
                
                # Extract video HD URL
                try:
                    results['video_hd_url'] = element[0]['snapshot']['cards'][0]['video_hd_url']
                except (KeyError, ValueError, IndexError):
                    results['video_hd_url'] = element[0]['snapshot'].get('videos', [{}])[0].get('video_hd_url', '')
                
                # Append extracted ad data to the list
                ads.append(results)
    except (KeyError, ValueError):
        continue
    
    entry_list.append(entry_counter)
    entry_counter += 1

# Convert extracted data into a Pandas DataFrame
df = pd.DataFrame.from_dict(ads)
df['order'] = df.index + 1  # Assign sequential order

# Calculate days the ad has been running
df['daysRunning'] = (pd.to_datetime(df['endDate']) - pd.to_datetime(df['startDate'])).dt.days

# Determine if ad is still running
maxend = max(pd.to_datetime(df['endDate']))
df['stillRunning'] = pd.to_datetime(df['endDate']) >= (maxend - timedelta(days=4))

# Set up Excel preview formulas
df['image_prev'] = '=image(' + xlsxwriter.utility.xl_col_to_name(df.columns.get_loc("original_image_url")) + df.index.astype(str) + ')'
df['vid_prev'] = '=image(' + xlsxwriter.utility.xl_col_to_name(df.columns.get_loc("video_preview_image_url")) + df.index.astype(str) + ')'

# Define export column order
export_cols = ['pageName', 'order', 'ad_link', 'daysRunning', 'image_prev', 'vid_prev', 'body_text',  'title', 'caption', 'link_description', 'link_url', 'cta_text', 'startDate', 'endDate', 'pageID', 'original_image_url', 'video_preview_image_url', 'video_hd_url', 'stillRunning', 'adArchiveID']
df = df[export_cols]

# Export data to CSV
df.to_csv(output_filename, index=False, encoding='utf-8-sig')

# Export data to Excel
df.to_excel(xlsx_filename, index=False, engine='xlsxwriter')
